## Challenge #2 Second Semester 2025 HPC ing mat, Turco Vale 10809855

I'll use my public [github repo](https://github.com/VTHVW/HPLabrynth). you can check that the last commit and push was made before the challenge deadline.

It builds a binary that handles both the labyrinth checking and creation.

First, we need to actually clone the repo and build the binaries. We are also going to need zsh, since i have also made a couple of zsh scripts that will generate a bunch of labyrinths for us automatically for testing.

In [ ]:
!sudo apt install zsh git

we start by cloning the repo and making the 3 zsh scripts executable.

In [ ]:
%cd /content/
!rm -rf HPLabrynth && git clone https://github.com/VTHVW/HPLabrynth.git
%cd /content/HPLabrynth/
!chmod +x ./compile* ./test_labs ./make_lots_of_labs

You should probaly read the README.md and the other files on github but i am also going to use "cat" here on colab to show the interesting files.

In [ ]:
!cat README.md

This explains part of the logic used for creating and checking the labyrinth.

The important class is `HPLab` in the `HPLabrynth` namespace.

Defined as follows:





In [ ]:
!tail src/HPLabrynth.h -n 27 | head -n 24 | sed '/operator/d' | sed '/^$/d'

##Labyrinth Generation
The labyrinth are generated with the second listed constructor, which can chose to either generate an always valid labyrinth or generate one without checking if it is. this is controlled by the `make_valid` parameter.
(by valid here i mean a labyrinth with a path between entrance and exit).

The generation is done with some simple PRNG, seeded by a `random_device` when present on the machine, and the current time otherwise.

The validity check is done with a simplifed version of the method described in this [wikipedia article](https://en.wikipedia.org/wiki/Maze_generation_algorithm#Iterative_randomized_Kruskal's_algorithm_(with_sets)).
This algorithm is far from perfect and sometimes it generate labyrinth with too few walls. It could be improved by repeating the generation until a certain amount of valid cells is obtained.

## Labyrinth Checking
To check a labyrinth we first need to retrieve the Labirynth parameters from a file, this is done with the `HPLAbrynth::getLabParamsFromFile` function. We can directly pass the output of this function to the HPLab constructor.

This will initialize the DS data structure, calling `makeSet` on each cell.
Note that every cell contains a `valid` field, this is set to false when we are on a wall ("x") and true when we are on a path (" "). Every operation of the DS data structure implemented (link,find and unite) does nothing when called on an invalid cell. This simplifies the class logic.

Finally to check if the labrynth is valid we can call `HPLab::hasPath`. This method will call `unite` on every set, linking the ones that are not separated by invalid cells (on which `link` does nothing). This method returns a bool which is simply `find(start) == find(end)`. Indeed the path is found if start and end are part of a connected graph (and therefore have the same set representative).


Note: N and M are the dimensions of the labyrinth (width and height respectively).

Next I'll show the most interesting methods (please go on github for syntax highlighting...).

## The DS methods:
(HPLabrynth.cpp line 331)

In [ ]:
!tail src/HPLabrynth.cpp  -n 65 | head -n 38 | sed '/operator/d' | sed '/^$/d'

## The constructors:
(HPLabrynth.cpp line 256)

In [ ]:
!tail src/HPLabrynth.cpp  -n 188 | head -n 64 | sed '/^$/d'

## Executing the code
Now we can compile the code. i am going to use my own script to do it, cmake was far too complicated for this simple project, therefore I'm just going to use the g++ linux command.

I've made two versions of the "compile" script: one just compiles with normal parameters, the other adds -O3. I haven't exensively checked the -O3 version for bugs but it should work without problems. (the executable name is also different)

Note: the library is statically linked.

In [ ]:
!cat ./compile
!./compile

## Finally using the program!
Now that it is compiled the executable offers us with a couple of options.

I've provided the program with a "help" option, to roughly explain its usage. This part of the libary is handled in the `HPOpts` namespace and won't be explained here because it wasn't part of what the challenge required. It is just a way to get the parameters from files, command line or user input and print the results on screen.

In [ ]:
!bin/HPLabrynth --help

## Checking Labs:
I've put some "test" files in the "files" directory.

We can use the --check option as follows to test those files.

(the output is the file itself plus a line with either "No path found" or "Has a path!!")

In [ ]:
!bin/HPLabrynth --check ./files/test_path

In [ ]:
!bin/HPLabrynth --check ./files/test_nopath

## Colors!!!

I've hadded a colorful mode to make the "walls" more visible for easier human checking. we can simply add --pretty to most combination of parameters.

Note that we can only save to file with the correct (x and ' ') format, and walls now surround the entire labyrinth.

In [ ]:
!bin/HPLabrynth --check ./files/test_path --pretty

In [ ]:
!bin/HPLabrynth --check ./files/test_nopath --pretty

## Generating a Labyrinth:

We can use the --rand option, this will output the generated labyrinth to std output, to save the result to file we can either use the --out option or just redirect the output to a file in the linux shell.

Note that unless the --skip option is provided the program will always generate a valid labyrinth.

To do so we can either pass the labyrinth parameters ourselves:

In [ ]:
!bin/HPLabrynth --rand="9 9 1 1 9 9"

or having the user input them: (separated by whitespaces)

In [ ]:
!bin/HPLabrynth --rand

Of course we can make this output colorful too: (but note that --out and -pretty are mutually exclusive options)

In [ ]:
!bin/HPLabrynth --rand="9 9 1 1 9 9" --pretty

Here we can save the result to a file and then check if it has indeed a valid path:

In [ ]:
!bin/HPLabrynth --rand="9 9 1 1 9 9" --out="./files/9x9randLab.txt"
!cat ./files/9x9randLab.txt
!bin/HPLabrynth --check ./files/9x9randLab.txt --pretty

## Skipping the checking:
This will generate a labyrinth with a 50% chance of walls per cell. (Therefore unlikely valid).

I could have probably set the RNG to 30% or 40% or even have had the user choose the threshold, but I elected not to do that.

We can do so with the --skip option. (For simplicity this option is not compatible with either --pretty or --out, please use the linux shell functionality to save the result to file.)

In [ ]:
!bin/HPLabrynth --rand="10 20 1 1 9 9" --skip

In [ ]:
!bin/HPLabrynth --rand="10 20 1 1 9 9" --skip > ./files/10x20randUnchekedLab.txt
!cat ./files/10x20randUnchekedLab.txt; echo -e "\n\n"
!bin/HPLabrynth --check ./files/10x20randUnchekedLab.txt --pretty

This concludes what was asked by the challenge.

## More Testing!

I've also made a couple of zsh scprits that use `HPLabrynth` to generate a bunch of different sized labyriths. This is just to test the program, nonetheless I have included them here so that you can test it too.

The scripts are very simple, here is the code:

In [ ]:
!cat ./make_lots_of_labs

In [ ]:
!cat ./test_labs

In [ ]:
!./make_lots_of_labs
!./test_labs